In [1]:
# %pip install tqdm
# %pip install pillow
# %pip install pandas
# %pip install pytest-plt
# %pip install openpyxl
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from algorithms.black_hole import BlackHole
from algorithms.simulated_annealing import SimulatedAnnealing
from algorithms.genetic_algorithm import GeneticAlgorithm
from algorithms.lagrangian_heuristics import LagrangianHeuristics
from algorithms.hybrid_approach import GA_SA
from algorithms.hybrid_approach import LH_SA
from algorithms.hybrid_approach import BH_SA
from algorithms.hybrid_approach import LH_BH
from algorithms.hybrid_approach import SA_GA
from services.generation import Test

In [4]:
def start_algorithm(alg, params, name, optimum=None):
    data = []
    for _ in range(5):
        if name[-5:] != 'BH_AD':
            results, times, solution = alg(*params, optimum=optimum)
        else:
            results, times, counts, solution = alg(*params, optimum=optimum)
            plt.plot(counts)
            plt.savefig(f'experiments/charts/{name + '_counts'}.png')
            plt.clf()
        data.append({'results': results, 'time': times})
    if len(data[0]['results']) == 2 and isinstance(data[0]['results'][0], list):
        data = min(data, key=lambda x: min(x['results'][0] + x['results'][1]))
        plt.plot(data['results'][0])
        bound = len(data['results'][0]) - 1
        plt.plot(list(range(bound, bound + len(data['results'][1]))), data['results'][1])
        plt.savefig(f'experiments/charts/{name}.png')
        plt.clf()
        print(min(data['results'][0]), min(data['results'][1]))
        data['results'] = data['results'][0] + data['results'][1]
    else:
        data = min(data, key=lambda x: (x['results'][-1], x['time'][x['results'].index(x['results'][-1])]))
        plt.plot(data['results'])
        plt.savefig(f'experiments/charts/{name}.png')
        plt.clf()
    result = min(data['results'])
    return result,  round(data['time'][data['results'].index(result)], 2)

In [5]:
def start_experiments(algorithms, files, myTest=False):
    frames = []
    for file in files:
        FILE = f'tests/{file['name']}.txt'
        test = Test()
        if myTest:
            test.load_generated_data(FILE)
        else:
            test.load_or_library_data(FILE)
        data = {'OPT': file['optimum']}

        for algorithm in algorithms:
            exp = algorithm['class'](test)
            print(file['name'] + ':', algorithm['name'])
            val, time = start_algorithm(exp.start, algorithm['params'], f'{file['name']}_{algorithm['name']}', 
                                        file['optimum'])
            data[f'{algorithm['name']} value'] = val
            data[f'{algorithm['name']} time'] = time
        print(data)
        frames.append(pd.DataFrame(data, index=[file['name']]))
    df = pd.concat(frames)
    df.to_excel(f'experiments/tables/{'-'.join([alg['name'] for alg in algorithms])}_{int(myTest)}.xlsx')  
    return df


In [ ]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

algorithms = [{'name': 'BH', 'params': [500, 20, False], 'class': BlackHole},
              {'name': 'BH_AD', 'params': [500, 20, True], 'class': BlackHole}]
start_experiments(algorithms, files)

In [6]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

algorithms = [ {'name': 'GA_ELS', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRS', 'params': [20, 0.5, (0, 100, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAS', 'params': [20, 0.5, (100, 0, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAPRS', 'params': [20, 0.5, (50, 50, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAELS', 'params': [20, 0.5, (50, 0, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRELS', 'params': [20, 0.5, (0, 50, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLS', 'params': [20, 0.5, (33, 33, 34), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files)

4: GA_ELS


100%|██████████| 50/50 [00:59<00:00,  1.20s/it]


4: GA_PRS


100%|██████████| 50/50 [00:56<00:00,  1.12s/it]


4: GA_RAS


...

6: GA_RAPRELS_50


100%|██████████| 50/50 [01:00<00:00,  1.22s/it]


{'OPT': 138, 'GA_ELS value': 167, 'GA_ELS time': 39.19, 'GA_PRS value': 176, 'GA_PRS time': 61.91, 'GA_RAS value': 163, 'GA_RAS time': 62.14, 'GA_RAPRS value': 165, 'GA_RAPRS time': 28.37, 'GA_RAELS value': 161, 'GA_RAELS time': 58.65, 'GA_PRELS value': 163, 'GA_PRELS time': 39.03, 'GA_RAPRELS_50 value': 156, 'GA_RAPRELS_50 time': 62.19}


,OPT,GA_ELS value,GA_ELS time,GA_PRS value,GA_PRS time,GA_RAS value,GA_RAS time,GA_RAPRS value,GA_RAPRS time,GA_RAELS value,GA_RAELS time,GA_PRELS value,GA_PRELS time,GA_RAPRELS_50 value,GA_RAPRELS_50 time
4,429,467,55.79,476,46.51,462,40.64,489,43.63,450,52.14,476,46.19,473,26.80
5,253,284,99.59,291,118.96,272,110.15,290,70.51,309,38.76,312,80.94,295,64.16
6,138,167,39.19,176,61.91,163,62.14,165,28.37,161,58.65,163,39.03,156,62.19


<Figure size 640x480 with 0 Axes>

In [7]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

algorithms = [ {'name': 'GA_ONC', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAC', 'params': [20, 0.5, (0, 0, 100), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 100, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_UNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 0, 100), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAONC', 'params': [20, 0.5, (0, 0, 100), (50, 50, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWUNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 50, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ONUNC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm}, 
               {'name': 'GA_ONTWC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RATWC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAUNC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLC', 'params': [20, 0.5, (0, 0, 100), (25, 25, 25, 25), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files)

4: GA_ONC


100%|██████████| 50/50 [00:32<00:00,  1.56it/s]


4: GA_RAC


100%|██████████| 50/50 [00:37<00:00,  1.33it/s]


4: GA_TWC


...

6: GA_ALLC


100%|██████████| 50/50 [00:32<00:00,  1.53it/s]

{'OPT': 138, 'GA_ONC value': 157, 'GA_ONC time': 31.72, 'GA_RAC value': 141, 'GA_RAC time': 12.83, 'GA_TWC value': 158, 'GA_TWC time': 48.84, 'GA_UNC value': 141, 'GA_UNC time': 25.79, 'GA_RAONC value': 140, 'GA_RAONC time': 7.89, 'GA_TWUNC value': 149, 'GA_TWUNC time': 34.92, 'GA_ONUNC value': 159, 'GA_ONUNC time': 28.68, 'GA_ONTWC value': 159, 'GA_ONTWC time': 35.84, 'GA_RATWC value': 141, 'GA_RATWC time': 9.26, 'GA_RAUNC value': 142, 'GA_RAUNC time': 7.13, 'GA_ALLC value': 142, 'GA_ALLC time': 6.79}


,OPT,GA_ONC value,GA_ONC time,GA_RAC value,GA_RAC time,GA_TWC value,GA_TWC time,GA_UNC value,GA_UNC time,GA_RAONC value,...,GA_ONUNC value,GA_ONUNC time,GA_ONTWC value,GA_ONTWC time,GA_RATWC value,GA_RATWC time,GA_RAUNC value,GA_RAUNC time,GA_ALLC value,GA_ALLC time
4,429,473,34.32,431,11.63,459,19.87,432,6.96,432,...,453,25.13,486,27.10,434,17.19,433,6.14,432,5.92
5,253,282,9.85,253,25.89,293,95.40,253,29.23,256,...,308,66.10,339,104.50,253,11.93,257,13.04,254,12.03
6,138,157,31.72,141,12.83,158,48.84,141,25.79,140,...,159,28.68,159,35.84,141,9.26,142,7.13,142,6.79


<Figure size 640x480 with 0 Axes>

In [ ]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [
               {'name': 'GA_PRS', 'params': [20, 0.5, (0, 100, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAS', 'params': [20, 0.5, (100, 0, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAPRS', 'params': [20, 0.5, (50, 50, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAELS', 'params': [20, 0.5, (50, 0, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRELS', 'params': [20, 0.5, (0, 50, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLS', 'params': [20, 0.5, (33, 33, 34), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

algorithms = [ {'name': 'GA_ONC', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAC', 'params': [20, 0.5, (0, 0, 100), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 100, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_UNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 0, 100), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAONC', 'params': [20, 0.5, (0, 0, 100), (50, 50, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWUNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 50, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ONUNC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm}, 
               {'name': 'GA_ONTWC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RATWC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAUNC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLC', 'params': [20, 0.5, (0, 0, 100), (25, 25, 25, 25), (0, 0), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)

In [8]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [ {'name': 'GA_0F', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_25F', 'params': [20, 0.25, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_50F', 'params': [20, 0.5, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_75F', 'params': [20, 0.75, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_100F', 'params': [20, 1, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

algorithms = [ {'name': 'GA_ELS', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files2, True)

test1: GA_0F


100%|██████████| 50/50 [00:00<00:00, 70.32it/s]


test1: GA_25F


100%|██████████| 50/50 [00:00<00:00, 66.22it/s]


test1: GA_50F


...

{'OPT': None, 'GA_ELS value': 1435, 'GA_ELS time': 0.81}
test4: GA_ELS


100%|██████████| 50/50 [00:14<00:00,  3.53it/s]


{'OPT': None, 'GA_ELS value': 664, 'GA_ELS time': 2.03}


,OPT,GA_ELS value,GA_ELS time
test1,None,899,0.10
test2,None,371,0.46
test3,None,1435,0.81
test4,None,664,2.03


<Figure size 640x480 with 0 Axes>

In [9]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [ {'name': 'GA_1000-25PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 25), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_1000-50PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 50), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_1000-75PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 75), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-25PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 25), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-50PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 50), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-75PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 75), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

test1: GA_1000-25PS


100%|██████████| 50/50 [00:00<00:00, 85.18it/s]


test1: GA_1000-50PS


100%|██████████| 50/50 [00:00<00:00, 97.66it/s]


test1: GA_1000-75PS


...

6: GA_500-75PS


100%|██████████| 50/50 [00:29<00:00,  1.69it/s]


{'OPT': 138, 'GA_1000-25PS value': 145, 'GA_1000-25PS time': 26.14, 'GA_1000-50PS value': 144, 'GA_1000-50PS time': 23.06, 'GA_1000-75PS value': 143, 'GA_1000-75PS time': 26.81, 'GA_500-25PS value': 143, 'GA_500-25PS time': 28.39, 'GA_500-50PS value': 143, 'GA_500-50PS time': 20.68, 'GA_500-75PS value': 146, 'GA_500-75PS time': 27.37}


,OPT,GA_1000-25PS value,GA_1000-25PS time,GA_1000-50PS value,GA_1000-50PS time,GA_1000-75PS value,GA_1000-75PS time,GA_500-25PS value,GA_500-25PS time,GA_500-50PS value,GA_500-50PS time,GA_500-75PS value,GA_500-75PS time
4,429,431,19.17,437,33.18,445,24.11,433,18.97,441,14.11,439,28.41
5,253,254,52.18,256,51.46,279,50.15,256,71.85,262,52.81,275,51.34
6,138,145,26.14,144,23.06,143,26.81,143,28.39,143,20.68,146,27.37


<Figure size 640x480 with 0 Axes>

In [10]:
algorithms = [{'name': 'GA_BEST', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 500], 
               'class': GeneticAlgorithm}]

files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

test1: GA_BEST


 29%|██▉       | 145/500 [00:04<00:10, 33.42it/s]


{'OPT': None, 'GA_BEST value': 899, 'GA_BEST time': 0.03}
test2: GA_BEST


 25%|██▌       | 127/500 [00:19<00:56,  6.64it/s]


{'OPT': None, 'GA_BEST value': 371, 'GA_BEST time': 0.46}
test3: GA_BEST


...

{'OPT': 179, 'GA_BEST value': 219, 'GA_BEST time': 354.69}
H: GA_BEST


  1%|          | 5/500 [05:17<8:43:15, 63.43s/it]


{'OPT': 64, 'GA_BEST value': 75, 'GA_BEST time': 233.39}


,OPT,GA_BEST value,GA_BEST time
4,429,430,39.74
5,253,254,110.45
6,138,141,20.79
A,253,254,151.55
B,69,69,168.85
C,227,230,281.02
D,60,61,81.61
E,29,29,153.28
F,14,15,17.53
G,179,219,354.69


<Figure size 640x480 with 0 Axes>

In [11]:
algorithms = [{'name': 'BH_AD', 'params': [500, 20, True], 'class': BlackHole}]
files= [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}]
start_experiments(algorithms, files, True)

test1: BH_AD


 21%|██        | 104/500 [00:04<00:16, 23.33it/s]


{'OPT': None, 'BH_AD value': 899, 'BH_AD time': 0.03}
test2: BH_AD


 22%|██▏       | 111/500 [00:16<00:58,  6.62it/s]


{'OPT': None, 'BH_AD value': 371, 'BH_AD time': 0.43}
test3: BH_AD


...

{'OPT': None, 'BH_AD value': 1435, 'BH_AD time': 0.34}
test4: BH_AD


 32%|███▏      | 162/500 [01:25<02:58,  1.89it/s]


{'OPT': None, 'BH_AD value': 664, 'BH_AD time': 1.0}


,OPT,BH_AD value,BH_AD time
test1,None,899,0.03
test2,None,371,0.43
test3,None,1435,0.34
test4,None,664,1.00


<Figure size 640x480 with 0 Axes>

In [6]:
algorithms = [{'name': 'LH', 'params': [5000], 'class': LagrangianHeuristics}]
files1 = [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}
         ]

files2 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

start_experiments(algorithms, files1, True)
start_experiments(algorithms, files2)

test1: LH


  0%|          | 0/5000 [00:00<?, ?it/s]


{'OPT': 899, 'LH value': 899, 'LH time': 0.01}
test2: LH


  0%|          | 20/5000 [00:00<00:38, 130.13it/s]


{'OPT': 371, 'LH value': 371, 'LH time': 0.15}
test3: LH


...

{'OPT': 179, 'LH value': 193, 'LH time': 236.0}
H: LH


  1%|          | 57/5000 [05:01<7:15:07,  5.28s/it]


{'OPT': 64, 'LH value': 68, 'LH time': 201.71}


,OPT,LH value,LH time
4,429,430,58.75
5,253,257,45.89
6,138,143,86.77
A,253,257,3.45
B,69,70,43.99
C,227,233,31.34
D,60,61,17.52
E,29,29,5.85
F,14,15,33.19
G,179,193,236.00


<Figure size 640x480 with 0 Axes>

In [8]:
algorithms = [{'name': 'SA', 'params': [500, 10000], 'class': SimulatedAnnealing}, ]
files1 = [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}]

files2 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

start_experiments(algorithms, files1, True)
start_experiments(algorithms, files2)

test1: SA


 10%|█         | 1014/10000 [00:00<00:03, 2274.45it/s]


{'OPT': 899, 'SA value': 899, 'SA time': 0.06}
test2: SA


100%|██████████| 10000/10000 [00:42<00:00, 237.41it/s]


{'OPT': 371, 'SA value': 371, 'SA time': 10.84}
test3: SA


...

{'OPT': 179, 'SA value': 306, 'SA time': 2.05}
H: SA


  3%|▎         | 295/10000 [04:58<2:43:56,  1.01s/it]


{'OPT': 64, 'SA value': 104, 'SA time': 2.07}


,OPT,SA value,SA time
4,429,439,175.26
5,253,307,267.87
6,138,152,183.70
A,253,398,165.94
B,69,107,218.28
C,227,381,231.68
D,60,100,234.86
E,29,44,0.67
F,14,18,0.59
G,179,306,2.05


<Figure size 640x480 with 0 Axes>

In [5]:
algorithms = [{'name': 'GA_SA', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 250, 500, 5000], 
               'class': GA_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

4: GA_SA


100%|██████████| 5000/5000 [01:59<00:00, 41.68it/s]


431 431
{'OPT': 429, 'GA_SA value': 431, 'GA_SA time': 27.87}
5: GA_SA


 56%|█████▋    | 2824/5000 [02:05<01:36, 22.54it/s]


256 256
{'OPT': 253, 'GA_SA value': 256, 'GA_SA time': 56.21}
6: GA_SA


...

247 245
{'OPT': 179, 'GA_SA value': 245, 'GA_SA time': 278.59}
H: GA_SA


  3%|▎         | 149/5000 [02:30<1:21:35,  1.01s/it]


82 82
{'OPT': 64, 'GA_SA value': 82, 'GA_SA time': 160.42}


,OPT,GA_SA value,GA_SA time
4,429,431,27.87
5,253,256,56.21
6,138,140,19.66
A,253,256,91.33
C,227,234,146.06
D,60,60,126.67
F,14,15,111.36
G,179,245,278.59
H,64,82,160.42


<Figure size 640x480 with 0 Axes>

In [17]:
algorithms = [{'name': 'LH_SA', 'params': [2500, 500, 10000], 
               'class': LH_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

4: LH_SA


 15%|█▌        | 1531/10000 [00:37<03:25, 41.16it/s]


430 430
{'OPT': 429, 'LH_SA value': 430, 'LH_SA time': 60.75}
5: LH_SA


 36%|███▌      | 3590/10000 [02:29<04:27, 23.93it/s]


257 257
{'OPT': 253, 'LH_SA value': 257, 'LH_SA time': 41.95}
6: LH_SA


...

194 194
{'OPT': 179, 'LH_SA value': 194, 'LH_SA time': 152.55}
H: LH_SA


  1%|▏         | 144/10000 [02:30<2:51:53,  1.05s/it]


69 69
{'OPT': 64, 'LH_SA value': 69, 'LH_SA time': 68.37}


,OPT,LH_SA value,LH_SA time
4,429,430,60.75
5,253,257,41.95
6,138,141,106.77
A,253,257,1.32
B,69,70,70.83
C,227,233,51.54
D,60,61,40.22
F,14,15,78.04
G,179,194,152.55
H,64,69,68.37


<Figure size 640x480 with 0 Axes>

In [16]:
algorithms = [{'name': 'BH_SA', 'params': [500, 20, 500, 5000], 
               'class': BH_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

4: BH_SA


100%|██████████| 5000/5000 [02:00<00:00, 41.65it/s]


433 432
{'OPT': 429, 'BH_SA value': 432, 'BH_SA time': 230.71}
5: BH_SA


 65%|██████▍   | 3243/5000 [02:29<01:21, 21.62it/s]


261 261
{'OPT': 253, 'BH_SA value': 261, 'BH_SA time': 79.22}
A: BH_SA


...

198 198
{'OPT': 179, 'BH_SA value': 198, 'BH_SA time': 164.97}
H: BH_SA


  3%|▎         | 138/5000 [02:30<1:28:25,  1.09s/it]


68 68
{'OPT': 64, 'BH_SA value': 68, 'BH_SA time': 114.06}


,OPT,BH_SA value,BH_SA time
4,429,432,230.71
5,253,261,79.22
A,253,256,133.76
C,227,234,40.65
G,179,198,164.97
H,64,68,114.06


<Figure size 640x480 with 0 Axes>

In [7]:
algorithms = [{'name': 'LH_BH', 'params': [5000, 500, 20], 
               'class': LH_BH}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

4: LH_BH


 25%|██▌       | 127/500 [04:29<13:12,  2.13s/it]


432 432
{'OPT': 429, 'LH_BH value': 432, 'LH_BH time': 22.04}
5: LH_BH


 23%|██▎       | 113/500 [04:33<15:36,  2.42s/it]


259 259
{'OPT': 253, 'LH_BH value': 259, 'LH_BH time': 29.65}
A: LH_BH


 14%|█▍        | 71/500 [04:32<27:24,  3.83s/it]


257 257
{'OPT': 253, 'LH_BH value': 257, 'LH_BH time': 2.12}
C: LH_BH


  1%|          | 44/5000 [00:30<57:15,  1.44it/s]


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
algorithms = [{'name': 'SA_GA', 'params': [500, 5000, 20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 250], 
               'class': SA_GA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)